In [182]:
from rl.dynamic_programming import V, S, A
from rl import dynamic_programming

In [183]:
from rl import markov_process, markov_decision_process

In [184]:
import operator
from typing import Mapping, Iterator, TypeVar, Tuple, Dict, Iterable

import numpy as np

from rl.distribution import Categorical, Choose
from rl.iterate import converged, iterate
from rl.markov_process import NonTerminal, State
from rl.markov_decision_process import (FiniteMarkovDecisionProcess,
                                        FiniteMarkovRewardProcess)
from rl.policy import FinitePolicy, FiniteDeterministicPolicy

In [185]:
from rl.midterm_2022.priority_q import  PriorityQueue
from rl.midterm_2022 import grid_maze

In [186]:
import importlib

In [187]:
from time import time

# Problem 1

In this problem we will implement the gaps-based value iteration algorithm mentioned in class.

The gaps-based iteration algorithm proceeds as follows

1. Initialize the value function to zero for all states: $v[s] = 0\ \forall s \in \mathcal{N}$
2. Calculate the gaps for each state: $g[s] = |v[s] - \max_a \mathcal{R}(s,a) + \sum_{s'} \mathcal{P}(s,a,s') \cdot v(s')|$
3. While there is some gap that exceeds a threshold
 - Select the state with the  largest gap: $s_{max} = \arg\max_{s \in \mathcal{N}} g[s]$
 - Update the value function for $s_{max}$: $v[s_{max}] = \max_a \mathcal{R}(s_{max},a) + \sum_{s'}\mathcal{P}(s_{max},a,s') \cdot v(s')$
 -  Update the gap for $s_{max}$: $g[s_{max}] = 0$
4. Return v

We will test your implementation on a grid maze MDP. We have defined this class in "grid_maze.py", you should  briefly familiarize yourself with that code. In particular pay attention to the difference in reward functions for the two classes "GridMazeMDP_Dense" and "GridMazeMDP_Sparse"

Here is how you can use the classes:

In [188]:
underlying_maze = grid_maze.Maze(10, 10)
maze_mdp = grid_maze.GridMazeMDP_Sparse(underlying_maze, 0, 0)

you can visualize the maze if you wish

In [189]:
print(maze_mdp)

--------------------
|*            | |   |
| + +-+ + +-+-+ +-+ +
| |   | |           |
| + +-+-+-+-+ + +-+ +
| |       |   |   | |
| +-+ + +-+-+-+-+ + +
|   | |         | | |
| + +-+ +-+-+ +-+-+ +
| |   |   |   | |   |
| +-+ + +-+-+ + + +-+
| | | |     |   |   |
| + + +-+ +-+-+-+ + +
| |   |         | | |
| +-+ + + + + +-+ +-+
| | | | | | |   |   |
| + +-+-+ +-+ + +-+ +
|     |     | |   | |
| + +-+-+ +-+-+-+-+ +
| |   |         |   |
|-+-+-+-+-+-+-+-+-+-+


you can also visualize a policy on the mdp

In [190]:
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 0.9)
print(maze_mdp.print_policy(v2_res[1]))

--------------------
|* < < < < < <|v|> v|
| + +-+ + +-+-+ +-+ +
|^|^ <|^|^ < < < < <|
| + +-+-+-+-+ + +-+ +
|^|^ < < <|> ^|^ <|^|
| +-+ + +-+-+-+-+ + +
|^ <|^|^ < < < <|^|^|
| + +-+ +-+-+ +-+-+ +
|^|^ <|^ <|> ^|v|> ^|
| +-+ + +-+-+ + + +-+
|^|v|^|^ < <|^ <|^ <|
| + + +-+ +-+-+-+ + +
|^|> ^|> ^ < < <|^|^|
| +-+ + + + + +-+ +-+
|^|v|^|^|^|^|^ <|^ <|
| + +-+-+ +-+ + +-+ +
|^ < <|> ^ <|^|^ <|^|
| + +-+-+ +-+-+-+-+ +
|^|^ <|> ^ < < <|> ^|
|-+-+-+-+-+-+-+-+-+-+


You will need to make use of the PriorityQueue class in your implementation. A PriorityQueue is an ordered queue which supports the following operations
1. isEmpty(self): check if the queue is empty   
2. contains(self, element): check if the queue contains an element
3. peek(self): peek at the highest priority element in the queue    
4. pop(self): remove and return the highest priority element in the queue    
5. insert(self, element, priority): insert an element into the queue with given priority
6. update(self, element, new_priority): update the priority of an element in the queue
7. delete(self, element): delete an element from the queue

Below are some examples of using the queue

In [191]:
q: PriorityQueue = PriorityQueue()
print(q.isEmpty(), ':', "the queue is empty")
q.insert("a", 1)
print(q.isEmpty(), ':',  "the queue is not empty")
print(q.contains("a"), ':',  "the queue contains a")
print(q.contains("b"), ':',  "the queue does not contain a")
print(q.peek(), ':',  "a is the first element in the queue")
q.insert("b", 0)
print(q.contains("b"), ':',  "the queue now contains b")
print(q.peek(), ':',  "b is now at the front of the queue")
x = q.pop()
print(x, ':',  "we removed b from the queue")
print(q.isEmpty(), ':',  "the queue still nonempty")
print(q.contains("a"), ':',  "the queue still contains a")
print(q.contains("b"), ':',  "the queue does not contain b anymore")
print(q.peek(), ':',  "a is at the front of the queue")
q.insert("c", 5)
print(q.peek(), ':',  "a is still at the front of the queue")
q.update("a", 6)
print(q.peek(), ':',  "after updating a is no longer at the front of the queue")

True : the queue is empty
False : the queue is not empty
True : the queue contains a
False : the queue does not contain a
(1, 'a') : a is the first element in the queue
True : the queue now contains b
(0, 'b') : b is now at the front of the queue
b : we removed b from the queue
False : the queue still nonempty
True : the queue still contains a
False : the queue does not contain b anymore
(1, 'a') : a is at the front of the queue
(1, 'a') : a is still at the front of the queue
(5, 'c') : after updating a is no longer at the front of the queue


### Part 1

In [192]:
from collections import defaultdict
def invert_transition_map(mdp: markov_decision_process.FiniteMarkovDecisionProcess[S, A]) ->\
            Mapping[S, Iterable[S]]:

    inverted_mapping: Dict[S, list[S]] = {}

    for s, d in mdp.mapping.items():
        for a, d1 in d.items():
            for (s1, r), p in d1:
                if (s1 in mdp.non_terminal_states) & (p > 0):
                    if (s1 in inverted_mapping):
                        inverted_mapping[s1] += [s]
                    else:
                        inverted_mapping[s1] = [s]

    return inverted_mapping



### Part 2

In [193]:
def gaps_value_iteration(
    mdp: markov_decision_process.FiniteMarkovDecisionProcess[S, A],
    gamma: float, 
    gaps: PriorityQueue) -> Iterator[V[S]]:

    v_0: V[S] = {s: 0.0 for s in mdp.non_terminal_states} #POINT1: initialize v
    dependency_map = invert_transition_map(mdp)
    
    def extended_vf(v: V[S], ss: State[S]) -> float:
        def non_terminal_vf(st: NonTerminal[S], v=v) -> float:
            return v[st]
        return ss.on_non_terminal(non_terminal_vf, 0.0)
    
    def Bst(v: V[S]) -> V[S]:
        return {s: max(mdp.mapping[s][a].expectation(
            lambda s_r: s_r[1] + gamma * extended_vf(v, s_r[0])
        ) for a in mdp.actions(s)) for s in v}
    
    def update(v: V[S]) -> V[S]:
        s0 = gaps.pop() #STEP1: pick the element with largest gap
        v[s0] = Bst(v)[s0] #STPE2: update the function value for s_max
        gaps.insert(s0, 0) #STEP3: update the gap fop s_max

        for s in dependency_map[s0]:
            gaps.update(s, abs(v[s] - Bst(v)[s])) #STEP4: update related states
        return v

    
    return iterate(update, v_0)


def gaps_value_iteration_result(
    mdp: FiniteMarkovDecisionProcess[S, A],
    gamma: float
) -> Tuple[V[S], FiniteDeterministicPolicy[S, A]]:
    
    gaps = PriorityQueue()

    DEFUALT_TOLERANCE = 1e-3
    
    
    def extended_vf(v: V[S], ss: State[S]) -> float:
        def non_terminal_vf(st: NonTerminal[S], v=v) -> float:
            return v[st]
        return ss.on_non_terminal(non_terminal_vf, 0.0)
    
    def Bst(v: V[S]) -> V[S]:
        return {s: max(mdp.mapping[s][a].expectation(
            lambda s_r: s_r[1] + gamma * extended_vf(v, s_r[0])
        ) for a in mdp.actions(s)) for s in v}
    
    v_0: V[S] = {s: 0.0 for s in mdp.non_terminal_states} #POINT1: initialize v
    for s in mdp.non_terminal_states:
        gaps.insert(s, abs(v_0[s] - Bst(v_0)[s])) #POINT2: initialize gaps
    
    def criterion(x: V[S], y: V[S]):
        return max(x[s] - y[s] for s in x) < DEFUALT_TOLERANCE

        
    opt_vf: V[S] = converged(
        gaps_value_iteration(mdp, gamma, gaps),
        done= criterion 
    )
        
    opt_policy: markov_decision_process.FiniteDeterministicPolicy[S, A] = dynamic_programming.greedy_policy_from_vf(
        mdp,
        opt_vf,
        gamma
    )

    return opt_vf, opt_policy

# Do not change the code below here, just run it

## Calculating the VF for a maze with sparse rewards

In [194]:
underlying_maze = grid_maze.Maze(50, 50)
maze_mdp = grid_maze.GridMazeMDP_Sparse(underlying_maze, 0, 0)

#### printing the runtime for the calculation

In [198]:
start = time()
v1_res = gaps_value_iteration_result(maze_mdp, 0.9)
print(time() - start)

46.052847146987915


In [199]:
start = time()
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 0.9)
print(time() - start)

0.7874865531921387


#### confirming that the value functions are identical

In [202]:
assert v1_res[1] == v2_res[1]

AssertionError: 

## Calculating the VF for a maze with dense rewards

In [171]:
maze_mdp = grid_maze.GridMazeMDP_Dense(underlying_maze, 0, 0)

#### printing the runtime for the calculation

In [172]:
start = time()
v1_res = gaps_value_iteration_result(maze_mdp, 1)
print(time() - start)

46.82037281990051


In [173]:
start = time()
v2_res = dynamic_programming.value_iteration_result(maze_mdp, 1)
print(time() - start)

2.4824483394622803


#### confirming that the value functions are identical

In [174]:
assert v1_res[1] == v2_res[1]

AssertionError: 